### Note: Data has been downloaded, validated and rescaled to 224,224 for images and body parts
***

### Imports
***

In [ ]:
import os
import shutil
### Deep Learning
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.random import set_seed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Arithmetic
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score
from skimage.transform import rescale, resize
from sklearn.utils import shuffle as shuffle_iterable
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

### Visuals
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
ROOT_FOLDER = './mura_data/MURA-v1.1'

### Set Random Seeds ###
***

In [ ]:
GLOBAL_SEED = 666 # The number of the Beast
np.random.seed(GLOBAL_SEED) 
set_seed(GLOBAL_SEED)

In [ ]:
def filenames(body_part, train=True):
    if train:
        csv_path=f"{ROOT_FOLDER}/train_image_paths_aug.csv"
    else:
        csv_path=f"{ROOT_FOLDER}/valid_image_paths_aug.csv"
    
    with open(csv_path, 'rb') as fin:
        d = fin.readlines()
        if body_part == 'all':
            imgs = [ROOT_FOLDER[:-9] + str(x, encoding='utf-8').strip() for x in d]
        else:
            imgs = [ROOT_FOLDER[:-9] + str(x, encoding='utf-8').strip() for x in d if
                            str(x, encoding='utf-8').strip().split('/')[2] == body_part]
    labels= [x.split('_')[-1].split('/')[0] for x in imgs]
    return imgs, labels

In [ ]:
BODY_PARTS = {
'XR_ELBOW' : 0,
'XR_FINGER' : 1,
'XR_FOREARM': 2,
'XR_HAND' : 3,
'XR_HUMERUS': 4,
'XR_SHOULDER': 5,
'XR_WRIST': 6
}

REV_BODY_PARTS = {BODY_PARTS[k]:k for k in BODY_PARTS}

### Explanatory Data Analysis ###

In [ ]:
### Read the Image files

In [ ]:
data = pd.read_csv(f"{ROOT_FOLDER}/train_image_paths_aug.csv")

### We will examine imbalance in 3 scenarios
* Label imbalance across training set
* Body part imbalance across training set
* Per Body part Label imbalance across training set

In [ ]:
pos_neg_count = pd.DataFrame(data['label'].value_counts()).reset_index().sort_values(by='label')
posneg_class_weights = compute_class_weight('balanced', classes=[0,1], y=data['label'].apply(lambda x: 1 if x=='positive' else 0).values)

body_part_count = pd.DataFrame(data.groupby('part').count()['img_path']).rename(columns={'img_path':'count'}).reset_index().sort_values(by='count')
body_part_class_weights = compute_class_weight('balanced', classes=[0,1,2,3,4,5,6], y=data['part'].values)

per_body_part_pos_neg_count = pd.DataFrame(data.groupby('part')['label'].value_counts()).rename(columns={'label':'count'}).reset_index()

XR_ELBOW_pos_neg = compute_class_weight('balanced',    classes=[0,1], y=data[data['part'] == 0]['label'].apply(lambda x: 1 if x=='positive' else 0).values)
XR_FINGER_pos_neg = compute_class_weight('balanced',   classes=[0,1], y=data[data['part'] == 1]['label'].apply(lambda x: 1 if x=='positive' else 0).values)
XR_FOREARM_pos_neg = compute_class_weight('balanced',  classes=[0,1], y=data[data['part'] == 2]['label'].apply(lambda x: 1 if x=='positive' else 0).values)
XR_HAND_pos_neg = compute_class_weight('balanced',     classes=[0,1], y=data[data['part'] == 3]['label'].apply(lambda x: 1 if x=='positive' else 0).values)
XR_HUMERUS_pos_neg = compute_class_weight('balanced',  classes=[0,1], y=data[data['part'] == 4]['label'].apply(lambda x: 1 if x=='positive' else 0).values)
XR_SHOULDER_pos_neg = compute_class_weight('balanced', classes=[0,1], y=data[data['part'] == 5]['label'].apply(lambda x: 1 if x=='positive' else 0).values)
XR_WRIST_pos_neg = compute_class_weight('balanced',    classes=[0,1], y=data[data['part'] == 6]['label'].apply(lambda x: 1 if x=='positive' else 0).values)

### Store imbalance score per part
***

In [ ]:
BODY_PART_IMB_SCORE = {
    'XR_ELBOW' :   {0:XR_ELBOW_pos_neg[0],    1:XR_ELBOW_pos_neg[1]},
    'XR_FINGER' :  {0:XR_FINGER_pos_neg[0],   1:XR_FINGER_pos_neg[1]},
    'XR_FOREARM':  {0:XR_FOREARM_pos_neg[0],  1:XR_FOREARM_pos_neg[1]},
    'XR_HAND' :    {0:XR_HAND_pos_neg[0],     1:XR_HAND_pos_neg[1]},
    'XR_HUMERUS':  {0:XR_HUMERUS_pos_neg[0],  1:XR_HUMERUS_pos_neg[1]},
    'XR_SHOULDER': {0:XR_SHOULDER_pos_neg[0], 1:XR_SHOULDER_pos_neg[1]},
    'XR_WRIST':    {0:XR_WRIST_pos_neg[0],    1:XR_WRIST_pos_neg[1]}
}

In [ ]:
BODY_PART_IMB_SCORE['XR_HUMERUS']

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Global Label Imbalance")
_ =sns.barplot(x="index", y="label", data=pos_neg_count)
plt.ylabel("Number of Examples")
plt.xlabel(None)
plt.xticks(ticks=[0,1], labels=['Positive', 'Negative'])
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Example Imbalance per Body Part")
_ =sns.barplot(x="part", y="count", data=body_part_count, order=body_part_count.part)
plt.ylabel("Number of Examples")
plt.xlabel(None)
plt.xticks(ticks=[f for f,_ in enumerate(REV_BODY_PARTS)], labels=[REV_BODY_PARTS[f] for f in body_part_count['part']])
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Label imbalance per Body Part")
ax = sns.barplot(x="part", y="count", hue="label", data=per_body_part_pos_neg_count, order=body_part_count.part)
plt.ylabel("Number of Examples")
plt.xlabel(None)
plt.xticks(ticks=[f for f,_ in enumerate(REV_BODY_PARTS)], labels=[REV_BODY_PARTS[f] for f in body_part_count['part']])
plt.show()
plt.close()

### Exersize Description

**Task**: Given a set of diagnostic images of a body part classify if there is a disease or not

**Data**:
- The data are grayscale images of varying size (usually (400-500) x (400-500))

**Approaches**
- According to the exersize the first approach would be to use a hand crafted CNN to classify the images.
- The second approach would be using a pretrained CNN for the same task.


**Preprocessing**
- Images will be normalized to MinMax or Imagenet scale.
- The effect of image augmentations will be tested as well.



In [ ]:
def split_stratified_into_train_val_test(df_input, stratify_colname,
                                         frac_train, frac_val,
                                         random_state=None, recombine=True):
    '''
    Splits a Pandas dataframe into two subsets (train, val)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column).
    '''

    if frac_train + frac_val  != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_val, y_train, y_val = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val)

    if recombine:
        return pd.concat([df_train,df_val],axis=0)
    return df_train, df_val

### Experiment 1: Focus on a specific Body Part
#### Note 1: Use the MobilNet_v2 architecture since it is lightweight and accurate
#### Note 2: We will use part of the training images as validation set, and the given validation set as test set
****

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 8
num_epochs = 100
image_size = 224  
patch_size = 16  
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
] 
transformer_layers = 4
mlp_head_units = [256, 128]

def create_vit_classifier():
    image_input = Input((224,224,3), name='CustomInputLayer')
    x = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(image_input)
    x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.05)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomZoom(-0.1, 0.1)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomHeight(0.01)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomWidth(0.01)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)(x)
    x = tf.keras.layers.experimental.preprocessing.Resizing(224,224)(x)
    x = Patches(patch_size=16)(x)
    encoded_patches = PatchEncoder(num_patches=14, projection_dim=64)(x)


    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = Flatten()(representation)
    representation = Dropout(0.5)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    decision = Dense(units=1, activation='sigmoid')(features)
    body_part = Dense(units=7, activation='softmax')(features)
    model = Model(inputs=inputs, outputs=[decision, body_part])
    return model

### I would like to use Residual Blocks so i will implement a simple residual block here
***

In [ ]:
def residual_block(x, downsample, filters, kernel_size):
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = BatchNormalization()(y)
    y = ReLU(negative_slope=0.1)(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = BatchNormalization()(out)
    out = ReLU(negative_slope=0.1)(out)
    return out

### This is a standard convolutional block
***

In [ ]:
def plain_block(x, pool, filters, kernel_size, use_bn, use_dr, strides):
    x = Conv2D(kernel_size=kernel_size, strides= strides, filters=filters, padding="same")(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = ReLU(negative_slope=0.1)(x)
    if use_dr > 0:
        x = Dropout(use_dr)(x)
    if pool > 0:
        x = MaxPool2D(pool_size=(pool,pool), strides=(pool,pool))(x)
    return x

### Added Weighted Binary Crossentropy to care for class imbalance
***

In [ ]:
def weighted_bincrossentropy(weights):
    weight_zero = tf.cast(weights[0], tf.float32)
    weight_one = tf.cast(weights[1], tf.float32)
    def _weighted_bincrossentropy(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        bin_crossentropy = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        weights_ = y_true * weight_one + (1. - y_true) * weight_zero
        weighted_bin_crossentropy = weights_ * bin_crossentropy 
        return tf.keras.backend.mean(weighted_bin_crossentropy)
    return _weighted_bincrossentropy

In [ ]:
def run_experiment(body_part, mode='pretrained'):
    experiment_name = body_part

    ### Gather the dataframe of the training images
    train_dataframe = pd.read_csv(ROOT_FOLDER + '/train_image_paths_aug.csv')
    train_dataframe = train_dataframe[train_dataframe['part'] == BODY_PARTS[experiment_name]]
    train_dataframe['label'] = train_dataframe['label'].apply(lambda x: 1 if x=='positive' else 0)


    ### Shuffle it before passing to a generator so classes are stratified
    if mode=='pretrained':
        train_dataframe = split_stratified_into_train_val_test(train_dataframe,stratify_colname='label',frac_train=0.9,frac_val=0.1,random_state=GLOBAL_SEED, recombine=True)
    else:
        train_dataframe = split_stratified_into_train_val_test(train_dataframe,stratify_colname='label',frac_train=0.9,frac_val=0.1,random_state=GLOBAL_SEED, recombine=True)

    ### Gather the dataframe of the test images
    test_dataframe = pd.read_csv(ROOT_FOLDER + '/valid_image_paths_aug.csv')
    test_dataframe = test_dataframe[test_dataframe['part'] == BODY_PARTS[experiment_name]]
    test_dataframe['label'] = test_dataframe['label'].apply(lambda x: 1 if x=='positive' else 0)

    ### Calculate the Dummy Baseline
    
    train_labels = train_dataframe['label'].values
    train_dummy_input = np.zeros_like(train_labels)

    test_labels = test_dataframe['label'].values
    test_dummy_input = np.zeros_like(test_labels)

    clf = DummyClassifier().fit(train_dummy_input, train_labels)
    dummy_train_prediction = clf.predict(train_dummy_input)
    dummy_test_prediction = clf.predict(test_dummy_input)

    print(f"Part: {experiment_name}")
    base_train_acc = accuracy_score(train_labels, dummy_train_prediction)
    base_cohen_score = cohen_kappa_score(train_labels, dummy_train_prediction)
    
    print(f"Baseline Train / Val Results: {base_train_acc} / {base_cohen_score}")

    base_test_acc = accuracy_score(test_labels, dummy_test_prediction)
    base_test_cohen_score = cohen_kappa_score(test_labels, dummy_test_prediction)
    print(f"Baseline Test Results: {base_test_acc} / {base_test_cohen_score}")

    # Base train / validation image generator
    if mode=='pretrained':
        datagen = ImageDataGenerator(
            validation_split=0.1,
            preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
            )

        # Base test image generator
        datagen2 = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    else:
        datagen = ImageDataGenerator(
            validation_split=0.1,
            rescale=1/255.0,
            )

        # Base test image generator
        datagen2 = ImageDataGenerator(rescale=1/255.0)




    train_generator = datagen.flow_from_dataframe(
        dataframe = train_dataframe,
        directory= ROOT_FOLDER[:-9],
        x_col = "img_path",
        y_col = "label",
        batch_size=32,
        seed=GLOBAL_SEED,
        shuffle=True,
        subset="training",
        validate_filenames=False,
        class_mode="raw",
        target_size=(224,224))


    validation_generator = datagen.flow_from_dataframe(
        dataframe = train_dataframe,
        directory= ROOT_FOLDER[:-9],
        x_col = "img_path",
        y_col = "label",
        batch_size=32,
        seed=GLOBAL_SEED,
        shuffle=False,
        subset="validation",
        validate_filenames=False,
        class_mode="raw",
        target_size=(224,224))

    test_generator = datagen2.flow_from_dataframe(
        dataframe = test_dataframe,
        directory= ROOT_FOLDER[:-9],
        x_col = "img_path",
        y_col = "label",
        batch_size=32,
        seed=GLOBAL_SEED,
        shuffle=False,
        validate_filenames=False,
        class_mode="raw",
        target_size=(224,224))

    ### Set the Training Callbacks
    es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=20, restore_best_weights=True)
    mc = ModelCheckpoint(filepath=f'./{experiment_name}/{mode}_2',monitor='val_accuracy', mode='max', save_best_only=True)
    rl = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.001, verbose=1, min_lr=1e-8, cooldown=5)
    tb = TensorBoard(f'./{experiment_name}/logs/{mode}_2', update_freq=1)
    callbacks = [es,rl]


    ### Get a pre-trained MobileNetV2 Model (or make a simple one from scratch) and the CohenKappa Metric
    if mode == 'pretrained':
        PreTrainedBase = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3))
    cohen_metric = tfa.metrics.CohenKappa(num_classes=2, sparse_labels=True)


    if mode == 'pretrained':
        # Freeze them up to the non-fc top layer
        for layer in PreTrainedBase.layers[:-1]:
            layer.trainable = False


    image_input = Input((224,224,3), name='CustomInputLayer')
    x = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(image_input)
    x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.05)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomZoom(-0.1, 0.1)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomHeight(0.01)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomWidth(0.01)(x)
    x = tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)(x)
    x = tf.keras.layers.experimental.preprocessing.Resizing(224,224)(x)

    ### Up to this point the layers would have been the same so differentiate the main model here
    if mode == 'pretrained':
        x = PreTrainedBase(x)
        x = Flatten()(x)
        x = Dense(units=128, activation='relu')(x)
        x = Dropout(0.2)(x)
        model_output = Dense(units=1, activation='sigmoid')(x)
        model= Model(inputs=image_input,outputs=model_output, name='all_model')
    else:
        ### Use my custom network
        x = plain_block(x, pool=2, filters=32,  kernel_size=(3,3),   strides=(1,1),  use_bn=True,   use_dr=0)
        x = plain_block(x, pool=2, filters=64,  kernel_size=(3,3),   strides=(1,1),  use_bn=False,   use_dr=0)
        x = plain_block(x, pool=2, filters=64,  kernel_size=(3,3),   strides=(1,1),  use_bn=True,   use_dr=0)
        x = plain_block(x, pool=2, filters=128,  kernel_size=(3,3),   strides=(1,1),  use_bn=False,   use_dr=0)

        x = Flatten()(x)
        x = Dense(units=128, activation='relu')(x)
        x = Dropout(0.2)(x)
        model_output = Dense(units=1, activation='sigmoid')(x)
        model= Model(inputs=image_input,outputs=model_output, name='all_model')

    model.compile(optimizer=Adam(lr=1e-4), loss=weighted_bincrossentropy(weights=BODY_PART_IMB_SCORE[experiment_name]), metrics=['accuracy', cohen_metric])
    model.summary()

    history = model.fit(train_generator,
                    steps_per_epoch = len(train_generator),
                    epochs = 30,
                    validation_data = validation_generator,
                    validation_steps = len(validation_generator),
                    callbacks=callbacks, workers=4, max_queue_size=20, use_multiprocessing=False)
    model.save(f'./{experiment_name}/{mode}_2')

    ### Write a Test function to update our logs
    def update_results(model, path, split='test'):
        if split == 'train':
            gen = train_generator
        elif split == 'valid':
            gen = validation_generator
        else:
            gen = test_generator
        l,a,c =model.evaluate(gen)
        if os.path.exists(path+f'/{split}_results.txt'):
            os.remove(path+f'/{split}_results.txt')
        with open(path+f'/{split}_results.txt', 'w') as fout:
            fout.write(f"{split} Loss,{split} Accuracy,{split} Cohen\n{l},{a},{c}")
        return

    ### Use it
    update_results(model, path=f'./{experiment_name}/logs/{mode}_2', split='test')
    update_results(model, path=f'./{experiment_name}/logs/{mode}_2', split='valid')
    update_results(model, path=f'./{experiment_name}/logs/{mode}_2', split='train') 

In [ ]:
# for i, body_part in enumerate(BODY_PARTS):
#     run_experiment(body_part, 'pretrained')
#     print()
#     print()
#     print(f"Finished {i+1}/7")

In [ ]:
for i, body_part in enumerate(BODY_PARTS):
    run_experiment(body_part, 'custom')
    print()
    print()
    print(f"Finished {i+1}/7")

### Write a function to clean up logs in case something bad happens
***

In [ ]:
def clean_logs(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    return

### Perform Experiments with Loaded Models for Reproducability
***

In [ ]:
def run_test_experiment(body_part, mode):
    ### Gather the dataframe of the test images
    test_dataframe = pd.read_csv(ROOT_FOLDER + '/valid_image_paths_aug.csv')
    test_dataframe = test_dataframe[test_dataframe['part'] == BODY_PARTS[body_part]]
    test_dataframe['label'] = test_dataframe['label'].apply(lambda x: 1 if x=='positive' else 0)
    if mode == 'custom':
        datagen2 = ImageDataGenerator(rescale=1/255.0)
    else:
        datagen2 = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    test_generator = datagen2.flow_from_dataframe(
        dataframe = test_dataframe,
        directory= ROOT_FOLDER[:-9],
        x_col = "img_path",
        y_col = "label",
        batch_size=32,
        seed=GLOBAL_SEED,
        shuffle=False,
        validate_filenames=False,
        class_mode="raw",
        target_size=(224,224))

    # Initialize the custom loss with balanced weights (it does not matter we are not going to train...)
    model = tf.keras.models.load_model(F'./{body_part}/{mode}', custom_objects={'_weighted_bincrossentropy': weighted_bincrossentropy([1,1])})
    # Predict per image
    l, a, c  = model.evaluate_generator(test_generator)
    with open(f'./{body_part}_{mode}_per_image.txt', 'w') as fout:
        fout.write(f"Test Loss, Test Accuracy, Test Cohen\n{l},{a},{c}")
    
    # Predict per_study
    predictions = model.predict(test_generator)
    predictions  = np.squeeze(predictions)
    test_dataframe['prediction'] = predictions
    test_dataframe['patient'] = test_dataframe['img_path'].apply(lambda x: x.split('/')[3])
    test_dataframe['study'] = test_dataframe['img_path'].apply(lambda x: x.split('/')[4].split('_')[0])
    study_dataframe = test_dataframe.groupby(['patient','study']).mean('prediction')
    study_dataframe['prediction'] = study_dataframe['prediction'].apply(lambda x: 1 if x > 0.499 else 0)
    y_true = study_dataframe['label'].values
    y_pred = study_dataframe['prediction'].values
    study_a = accuracy_score(y_true, y_pred)
    study_c = cohen_kappa_score(y_true, y_pred)
    with open(f'./{body_part}_{mode}_per_study.txt', 'w') as fout:
        fout.write(f"Test Accuracy, Test Cohen\n{study_a},{study_c}")
    


In [ ]:
# for i, body_part in enumerate(BODY_PARTS):
#     if body_part == 'XR_WRIST':
#      run_test_experiment(body_part, mode='pretrained')

In [ ]:
for i, body_part in enumerate(BODY_PARTS):
    run_test_experiment(body_part, mode='custom')